In [1]:
import numpy as np

In [2]:
def binomial_tree_option_pricing(S, K, T, r, sigma, n, option_type="call", exercise_type="european"):
    """
    Binomial Tree Option Pricing Model for European and American options.

    Parameters:
        S (float): Current stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate (annualized).
        sigma (float): Volatility of the underlying asset.
        n (int): Number of time steps.
        option_type (str): "call" or "put" (default: "call").
        exercise_type (str): "european" or "american" (default: "european").

    """
    # Step size
    dt = T / n

    # Up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u

    # Risk-neutral probability
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize stock price tree
    stock_price = [[0] * (i + 1) for i in range(n + 1)]
    for i in range(n + 1):
        for j in range(i + 1):
            stock_price[i][j] = S * (u ** j) * (d ** (i - j))

    # Initialize option price tree
    option_price = [[0] * (i + 1) for i in range(n + 1)]

    # Compute option values at maturity
    for j in range(n + 1):
        if option_type == "call":
            option_price[n][j] = max(0, stock_price[n][j] - K)
        elif option_type == "put":
            option_price[n][j] = max(0, K - stock_price[n][j])

    # Backward induction to calculate option price at t=0
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            # Discounted expected value of option price in the next step
            expected_value = np.exp(-r * dt) * (
                p * option_price[i + 1][j + 1] + (1 - p) * option_price[i + 1][j]
            )

            if exercise_type == "american":
                # For American options, check early exercise
                if option_type == "call":
                    early_exercise = max(0, stock_price[i][j] - K)
                elif option_type == "put":
                    early_exercise = max(0, K - stock_price[i][j])
                option_price[i][j] = max(expected_value, early_exercise)
            else:
                # For European options, no early exercise
                option_price[i][j] = expected_value

    # Option price at t=0
    return option_price[0][0]

In [3]:
# Parameters
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to maturity in years
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Volatility
n = 50  # Number of steps

# European Call Option
european_call_price = binomial_tree_option_pricing(S, K, T, r, sigma, n, option_type="call", exercise_type="european")
print(f"European Call Option Price: {european_call_price:.2f}")

# European Put Option
european_put_price = binomial_tree_option_pricing(S, K, T, r, sigma, n, option_type="put", exercise_type="european")
print(f"European Put Option Price: {european_put_price:.2f}")

# American Call Option
american_call_price = binomial_tree_option_pricing(S, K, T, r, sigma, n, option_type="call", exercise_type="american")
print(f"American Call Option Price: {american_call_price:.2f}")

# American Put Option
american_put_price = binomial_tree_option_pricing(S, K, T, r, sigma, n, option_type="put", exercise_type="american")
print(f"American Put Option Price: {american_put_price:.2f}")


European Call Option Price: 10.41
European Put Option Price: 5.53
American Call Option Price: 10.41
American Put Option Price: 6.07
